<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_reportC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML reportC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from scipy.stats import multivariate_normal

# scikit-learn のいろいろ
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

---
## GMMによる生成モデルを用いた2次元データの識別
---

2次元2クラスのデータのクラスごとの分布に GMM を当てはめて識別する実験をやってみよう．

In [ ]:
# データの生成
moonX, moonY = make_moons(n_samples=200, noise=0.25, random_state=5963)
xmin, xmax = -2, 3
ymin, ymax = -2.5, 2.5
NL = len(moonX)
K = 2

# グラフを描く
fig, ax = plt.subplots()
ax.scatter(moonX[moonY==0, 0], moonX[moonY==0, 1])
ax.scatter(moonX[moonY==1, 0], moonX[moonY==1, 1])
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

次のコードセルを実行すると，クラスごとのデータに GMM を当てはめる．
- 変数 `covtype` は，GMM を構成する正規分布の分散共分散行列の形を指定する．詳細は [sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) 参照．
- 変数 `M` は，GMM の正規分布の数を指定する．

In [ ]:
covtype = 'full'
M = 2

py = np.empty(K)
gmm = np.empty(K, dtype=object)
for k in range(K):
    py[k] = np.sum(moonY == k) / NL
    gmm[k] = GaussianMixture(n_components=M, covariance_type=covtype)
    gmm[k].fit(moonX[moonY == k, :])

次のコードセルを実行すると，推定された GMM の正規分布をデータに重ねて表示（左）するとともに，その GMM を用いて得られるクラス事後確率の値を可視化する（右）．

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:(xmax-xmin)/100, ymin:ymax:(ymax-ymin)/100]
X_mesh = np.dstack((x_mesh, y_mesh))

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    logp = gmm[k].score_samples(X_mesh.reshape((-1, 2)))
    p[k, :] = py[k] * np.exp(logp)
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(figsize=(9, 6))
cmap = ['Blues', 'Oranges', 'Greens']

# Gaussian を当てはめた結果
colors= ['blue', 'orange', 'green']
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    for m in range(M):
        mu = gmm[k].means_[m, :]
        if covtype == 'spherical':
            cov = gmm[k].covariances_[m] * np.eye(2)
        else:
            cov = gmm[k].covariances_[m, ::]
        ax0.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu, cov=cov), colors=colors[k], alpha=0.5)

ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

#### 問題1

`covtype` を `'full'`, `'diag'`, `'spherical'` の3通りで変えるとともに `M` を適当に変えて実験を行い，結果と考察をこのセルに書き込みなさい．
`covtype` を変えると何がどのように変わるのかも記すこと．


---
## GMMによる生成モデルを用いた手書き数字画像の識別
---

以前も用いたことのある，MNIST という手書き数字画像データセットの一部を用いる．

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw[:20000] / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw[:20000].astype(int) # クラスラベル．0 から 9 の整数値

# 学習データとテストデータの分割
XL, XT, yL, yT = train_test_split(Xall, yall, test_size=4000, random_state=4649, stratify=yall)
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)
NL, NT = len(XL), len(XT)

K = 10

---
### 実験1

次のセルを実行すると，クラスごとのデータに mixture 数（混合数，用いる正規分布の数） 4 の GMMを当てはめて，推定されたパラメータの一部を可視化する．

In [ ]:
M = 4

fig, ax = plt.subplots(M, K, figsize=(8, 8/K*M))
for k in range(K):
    gmm = GaussianMixture(n_components=M, covariance_type='full')
    gmm.fit(XL[yL == k, :])
    for m in range(M):
        img = gmm.means_[m, :].reshape((28, 28))
        ax[m, k].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[m, k].axis('off')

plt.show()

#### 問題2

この図がどんなパラメータを可視化したものになっているのかをこのセルに書き込みなさい．

---
### 実験2

次の2つのコードセルを実行すると，クラスごとのデータに mixture 数 `M` の GMM を当てはめ，それを用いて学習データとテストデータを識別させたときの正答率を算出する．

In [ ]:
# 学習
M = 2
py = np.empty(K)
gmm = np.empty(K, dtype=object)
for k in range(K):
    print(f'##### class{k} #####')
    py[k] = np.sum(yL == k) / NL  ### (a)
    gmm[k] = GaussianMixture(n_components=M, covariance_type='full', verbose=2, verbose_interval=1)
    gmm[k].fit(XL[yL == k, :])
    print()

In [ ]:
# 学習データの識別
LL = np.empty((NL, K))
for k in range(K):
    LL[:, k] = gmm[k].score_samples(XL) + np.log(py[k])
yL_pred = np.argmax(LL, axis=1)
countL = np.sum(yL_pred == yL)

# テストデータの識別
LT = np.empty((NT, K))
for k in range(K):
    LT[:, k] = gmm[k].score_samples(XT) + np.log(py[k])  ### (b)
yT_pred = np.argmax(LT, axis=1)   ### (c)
countT = np.sum(yT_pred == yT)

# 結果の表示
print(f'M = {M}   train: {countL}/{NL} = {countL/NL:.3f}   test: {countT}/{NT} = {countT/NT:.3f}')

#### 問題3

上記コードセル中の `### (a)`, `### (b)`, `### (c)` というコメントがついた行がそれぞれ何を計算しているかを考え，このコードセルに書き込みなさい．

次の問題のために，以下のことをやりなさい：

1. 実験2のコードセルを， `M` を 1 から 5 まで変えて実行し，得られた識別率の値をこの notebook 中に適当な形式でメモしておく（注）．
1. `GaussianMixture` の `covariance_type` を `'diag'` にして ↑ と同様のことを行う．ただし， `M` は 1 から 5 に限らず，より大きな値も試そう．
1. `GaussianMixture` の `covariance_type` を `'spherical'` にして...以下同文．

注: EMアルゴリズムによる GMM の当てはめでは，推定されるパラメータの値は，その初期値の選び方によって
変わります．そのため，一つの実験条件でも実験を複数回行うのが適切です．
しかし，計算量が多く実験に時間がかかるため，ここでは一つの実験条件での GMM の当てはめは一度だけでよいものとします．

#### 問題4

↑で行った実験の結果を示し，考察を記しなさい．必要なら複数のテキストセルを作ったらよい．